In [1]:
import pandas as pd
import numpy as np
import quandl
# ! pip install wrds

In [15]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## Theory of implementation

The ATM skew exhibits a power law decay at short maturities, with an exponent determined by H. 

$\sigma_{BS}(k, \tau)$ for the Black-Scholes implied volatility of an option with time-to-maturity $\tau$ and log-moneyness $k = \log(K/S)$, where $K$ is the option's strike price and $S$ is the current level of the underlying. The ATM skew at maturity $\tau$ is given by

$$\phi(\tau) = \left| \frac{\partial \sigma_{BS}(k, \tau)}{\partial k} \right|_{k=0}$$



$$\phi(\tau) \approx \text{constant} \times \tau^{H-1/2}, \quad \text{as } \tau \downarrow 0.$$

In other words, the ATM skew exhibits a power law decay at short maturities, with an exponent determined by $H$.


## Methodology

To estimate the parameter $H$ from market data:

1. For each day and each stock, use filtered option data
2. Fit a cubic spline to model implied volatility as a function of log(K/S)
3. Calculate the derivative of the spline at log(K/S) = 0 to obtain the ATM skew ($\phi$)
4. Run a linear regression in log-log space:
  $$\log(\phi) = c + (H - \frac{1}{2})\log(\tau) + \varepsilon$$
5. Add $\frac{1}{2}$ to the estimated slope coefficient to determine the implied Hurst parameter $H$

This approach allows for robust estimation of the roughness parameter $H$ across different assets and time periods.


## Data import and cleaning

In [3]:
PATH_OPTIONS_DATA_1 = r"industry1_6_option.csv"
PATH_OPTIONS_DATA_2 = r"industry6_11.csv"

In [4]:
options_data_1_6 = pd.read_csv(PATH_OPTIONS_DATA_1)
options_data_1_6.head()

,secid,date,days,impl_volatility,impl_strike,cp_flag,ticker,index_flag
0,100972,2010-01-04,10,NaN,0.0,P,ABT,0
1,100972,2010-01-04,10,NaN,0.0,P,ABT,0
2,100972,2010-01-04,10,NaN,0.0,P,ABT,0
3,100972,2010-01-04,10,NaN,0.0,P,ABT,0
4,100972,2010-01-04,10,NaN,0.0,P,ABT,0


## Data Import Process

To fetch options data from OptionsMetrics on WRDS:

1. Connect to WRDS database using the wrds-python package
2. Query the OptionsMetrics database (optionm.opprcd table) to get:
   - Options data from Jan 2010 to Dec 2023
   - Data for specific major tech stocks (AAPL, GOOGL, MSFT, AMZN, META)
3. Convert date fields to datetime format for further processing

The following PostGreSQL queries were used to fetch TAQ and OptionMetrics data from WRDS:

In [ ]:
# sql = f"""
#   SELECT 
#       v.date,
#       v.days,
#       v.impl_volatility,
#       v.impl_strike,
#       v.cp_flag,
#       (SELECT n.ticker 
#        FROM optionm_all.optionmnames n 
#        WHERE n.secid = v.secid 
#        LIMIT 1) AS ticker
#   FROM optionm_all.vsurfd{year} v
#   WHERE EXISTS (
#       SELECT 1
#       FROM optionm_all.optionmnames n 
#       WHERE n.secid = v.secid 
#         AND n.ticker IN {tickers_str}
#   )
#   """

The query below is to fetch WRDS OptionMetrics data:


In [ ]:

# sql = f"""
#     WITH windowable_nbbo AS (
#         SELECT
#             sym_root AS ticker,
#             date,
#             time_m,
#             time_m_nano,
#             sym_root,
#             qu_cond,
#             best_bid,
#             best_bidsizeshares,
#             best_ask,
#             best_asksizeshares,
#             EXTRACT(HOUR FROM time_m) AS hour_of_day,
#             EXTRACT(MINUTE FROM time_m) AS minute_of_hour,
#             ROW_NUMBER() OVER (
#                 PARTITION BY sym_root, EXTRACT(HOUR FROM time_m), EXTRACT(MINUTE FROM time_m) 
#                 ORDER BY time_m DESC
#             ) AS rownum
#         FROM taqm_{year_str}.complete_nbbo_{date_str} 
#         WHERE sym_root IN {tickers}
#         AND sym_suffix IS NULL
#         AND time_m > '09:30:00' 
#         AND time_m < '16:00:00'
#     )
#     SELECT DISTINCT ON (ticker, date, hour_of_day, minute_of_hour)
#         ticker,
#         date,
#         date + DATE_TRUNC('minute', time_m) + INTERVAL '1 minute' AS window_time,
#         best_bid,
#         best_bidsizeshares,
#         best_ask,
#         best_asksizeshares,
#         time_m AS time_of_last_quote,
#         time_m_nano AS time_of_last_quote_ns
#     FROM windowable_nbbo
#     WHERE rownum = 1
#     """

Saved in csv after pulling the data

The current implementation is designed to process a subset of Options Data 
The goal is to optimize the code so that it can efficiently fetch, process, and analyze options data for multiple tickers across multiple years without excessive computational overhead.

In [5]:
options_data_1_6['date'] = pd.to_datetime(options_data_1_6['date'])

In [6]:
options_data_1_6['expiration'] = options_data_1_6['date'] + pd.to_timedelta(options_data_1_6['days'], unit='D')

In [ ]:
# options_6_11 = pd.read_csv(PATH_OPTIONS_DATA_2)
# options_6_11.head()
# options_6_11['date'] = pd.to_datetime(options_6_11['date'])
# options_6_11['expiration'] = options_6_11['date'] + pd.to_timedelta(options_6_11['days'], unit='D')

In [10]:
import os

def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.environ["HOME"], "Downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key="TDynpHNbX5ucpahcz_85"
, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

In [11]:
prices_eod_primary = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=False)
prices_eod_primary['date'] = pd.to_datetime(prices_eod_primary['date'])
prices_eod_primary.head()

Data file /Users/mahimaraut/Downloads/QUOTEMEDIA/PRICES_20250301.zip size 1674608027 exists already, no need to download


,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,JTKWY,2022-03-11,6.1700,7.3200,5.7900,6.72,9440097.0,0.0,1.0,6.1700,7.3200,5.7900,6.72,9440097.0
1,JTKWY,2022-03-10,6.1600,6.1750,5.9350,6.07,2261623.0,0.0,1.0,6.1600,6.1750,5.9350,6.07,2261623.0
2,FG_1,2020-06-01,8.1000,8.3900,8.1000,8.39,3086317.0,0.0,1.0,8.1000,8.3900,8.1000,8.39,3086317.0
3,YTENQ,2024-09-30,0.9514,1.0500,0.9514,1.05,842.0,0.0,1.0,0.9514,1.0500,0.9514,1.05,842.0
4,FLWS,2022-03-09,14.5700,14.9588,14.4100,14.45,662492.0,0.0,1.0,14.5700,14.9588,14.4100,14.45,662492.0


In [23]:
industries = pd.read_csv('industry.csv')

In [24]:
industries.head()

,industry,top_tickers
0,CRSP US Consumer Discretionary Index,AMZN TSLA HD WMT MCD BKNG LOW TJX UBER...
1,CRSP US Consumer Staples Index,COST PG KO PEP PM MO MDLZ CL KMB KVUE
2,CRSP US Energy Index,XOM CVX COP EOG WMB OKE SLB KMI PSX LNG
3,CRSP US Financials Index,BRK.A/BRK.B JPM V MA BAC WFC GS SPGI M...
4,CRSP US Healthcare Index,LLY UNH JNJ ABBV MRK TMO ABT ISRG PFE ...


In [25]:
industries['industry'] = industries.industry.str.replace('CRSP ', '', regex=True)
industries['industry'] = industries.industry.str.replace(' Index', '', regex=True)
industries['top_tickers'] = industries['top_tickers'].str.split()
industries = industries.explode('top_tickers').reset_index(drop=True)
industries

,industry,top_tickers
0,US Consumer Discretionary,AMZN
1,US Consumer Discretionary,TSLA
2,US Consumer Discretionary,HD
3,US Consumer Discretionary,WMT
4,US Consumer Discretionary,MCD
...,...,...
105,US Utilities,AEP
106,US Utilities,VST
107,US Utilities,D
108,US Utilities,PCG


In [26]:
industries.rename(columns={'top_tickers': 'tickers'}, inplace=True)
industries

,industry,tickers
0,US Consumer Discretionary,AMZN
1,US Consumer Discretionary,TSLA
2,US Consumer Discretionary,HD
3,US Consumer Discretionary,WMT
4,US Consumer Discretionary,MCD
...,...,...
105,US Utilities,AEP
106,US Utilities,VST
107,US Utilities,D
108,US Utilities,PCG


In [16]:
ticker_list = industries['tickers'].tolist()


In [ ]:

options_dict = {ticker : options_dict[options_dict['ticker'] == ticker] for ticker in ticker_list if ticker in options_dict}

In [22]:
row = options_dict.keys()
row


dict_keys(['A', 'ABBV', 'ABT', 'AMZN', 'AXP', 'B', 'BAC', 'BKNG', 'BRK', 'CL', 'COP', 'COST', 'CVX', 'EOG', 'GS', 'HD', 'JNJ', 'JPM', 'KMB', 'KMI', 'KO', 'KVUE', 'LLY', 'LNG', 'LOW', 'MA', 'MCD', 'MDLZ', 'MO', 'MRK', 'MS', 'OKE', 'PEP', 'PG', 'PM', 'PSX', 'SBUX', 'SLB', 'SPGI', 'TJX', 'TMO', 'TSLA', 'UBER', 'UNH', 'V', 'WFC', 'WMB', 'WMT', 'XOM'])

In [26]:
def power_law(tau, C, H):
    """ Power-law function: phi(tau) = C * tau^(H - 1/2) """
    return C * tau**(H - 0.5)

In [27]:
def get_atm_skew_slope(ticker, date, stock_price, options_data, method='mid', flag_plot=False):
    # Create a copy of options_data to avoid modifying the original DataFrame
    options_data = options_data.copy()
    
    # Use .loc[] for assignment to avoid ambiguity
    options_data.loc[:, 'impl_strike'] = pd.to_numeric(options_data['impl_strike'], errors='coerce')
    options_data.loc[:, 'impl_volatility'] = pd.to_numeric(options_data['impl_volatility'], errors='coerce')

    # Rest of your code remains the same...
    
    # Separate calls and puts
    call_options = options_data[options_data['cp_flag'] == 'C'].dropna(subset=['impl_volatility'])
    put_options = options_data[options_data['cp_flag'] == 'P'].dropna(subset=['impl_volatility'])

    # Compute log-moneyness for calls and puts
    call_options['log(K/S)'] = np.log(call_options['impl_strike'].astype(float) / stock_price['close'].iloc[0])
    put_options['log(K/S)'] = np.log(put_options['impl_strike'].astype(float) / stock_price['close'].iloc[0])

    # Sort data for proper spline fitting
    call_options = call_options.sort_values(by='log(K/S)')
    put_options = put_options.sort_values(by='log(K/S)')

    # Extract values
    log_ks_calls = call_options['log(K/S)'].values
    vols_calls = call_options['impl_volatility'].values
    
    log_ks_puts = put_options['log(K/S)'].values
    vols_puts = put_options['impl_volatility'].values

    # Fit cubic splines separately for calls and puts
    spline_calls = CubicSpline(log_ks_calls, vols_calls)
    spline_puts = CubicSpline(log_ks_puts, vols_puts)

    # Define a common grid around ATM
    grid = np.linspace(-0.2, 0.2, 100)

    # Evaluate both splines on the grid and take their mid implied volatility values
    mid_iv = (spline_calls(grid) + spline_puts(grid)) / 2

    # Fit a cubic spline to the mid implied volatility curve
    spline_mid = CubicSpline(grid, mid_iv)

    # Compute the ATM skew slope (first derivative at log(K/S) = 0)
    atm_skew_slope = spline_mid.derivative()(0).item()

    return atm_skew_slope

In [28]:

def get_time_to_expiry(expiration, date):
    """Compute time to expiry in years, ensuring both dates are Timestamp objects."""
    expiration = pd.Timestamp(expiration)  # Ensure it's a Timestamp
    date = pd.Timestamp(date)  # Ensure it's a Timestamp

    return (expiration - date).days / 365.0  # Convert days to years

In [13]:
from joblib import Parallel, delayed
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt



def get_h_value(ticker, date, stock_price, options_data, flag_plot_image=False):
    """
    Calculate the H value (Hurst parameter) from options data using power law fitting.

    Args:
        ticker (str): Stock ticker symbol
        date (datetime): Current date
        stock_price (pd.DataFrame): Stock price data
        options_data (pd.DataFrame): Options data
        flag_plot_image (bool): Whether to plot the power law fit

    Returns:
        float or None: Optimized H value (None if insufficient data)
    """
    # Filter valid expirations (excluding current date)
    unique_expirations = options_data.loc[options_data['expiration'] != date, 'expiration'].unique()
    
    if len(unique_expirations) == 0:
        print(f"No valid expiration dates found for {ticker} on {date}.")
        return None

    expiration_phi_pairs = []

    # Process each expiration date
    for expiration in unique_expirations:
        print(f'Processing: Ticker={ticker}, Date={date}, Expiration={expiration}')
        
        # Filter options for the current expiration
        options_filtered = options_data[options_data['expiration'] == expiration]
        
        if options_filtered.empty:
            continue  # Skip if no options data available for this expiration
        
        # Calculate phi and time to expiry
        phi = get_atm_skew_slope(ticker, date, stock_price, options_filtered, "mid", False)
        
        # Check if phi is valid
        if phi is None:
            continue
        
        time_to_expiry = options_filtered['days'].iloc[0] / 365  # Convert days to years
        expiration_phi_pairs.append((time_to_expiry, phi))

    # Ensure we have valid pairs
    if len(expiration_phi_pairs) == 0:
        print(f"Insufficient data to fit power law for {ticker} on {date}.")
        return None

    # Convert list to NumPy arrays
    expirations, phis = zip(*expiration_phi_pairs)
    expirations = np.array(expirations)
    abs_phis = np.abs(phis)

    # Fit power-law model
    try:
        popt, _ = curve_fit(power_law, expirations, abs_phis, p0=(1, 0.5), maxfev=10000)
        C_opt, H_opt = popt
        fitted_phi = power_law(expirations, C_opt, H_opt)
    except RuntimeError:
        print(f"Curve fitting failed for {ticker} on {date}. Returning None.")
        return None

    # Plot the power-law fit if enabled
    if flag_plot_image:
        plt.figure(figsize=(10, 6))
        plt.plot(expirations, abs_phis, marker='o', linestyle='-', color='b', label="|Phi| vs Expiration")
        plt.plot(expirations, fitted_phi, linestyle="--", color="r", label="Fitted Power-Law Curve")
        
        plt.xlabel("Time to Expiration (years)")
        plt.ylabel("|Phi|")
        plt.title(f"Power-Law Fit: |Phi| vs Expiration for {ticker} on {date}")
        plt.legend()
        plt.grid(True)
        plt.show()

    print(f"Optimized H Value for {ticker} on {date}: {H_opt}")
    return H_opt


In [ ]:
import pandas as pd
import pandas_market_calendars as mcal
from joblib import Parallel, delayed
import numpy as np

# Get the trading calendar for NYSE
nyse = mcal.get_calendar('NYSE')

def process_ticker_year(ticker, year):
    # Get trading dates for the specified year
    trading_dates = nyse.valid_days(start_date=f'{year}-01-01', end_date=f'{year}-12-31')
    series = pd.Series(trading_dates.to_pydatetime())  # Convert to a Series
    series = series.dt.tz_localize(None).dt.normalize()  # Ensure it's timezone naive

    h_values_list = []

    for date in series:
        try:
            print(f'Ticker: {ticker}, Date: {date}')
            options_data = options_dict[ticker][options_dict[ticker]['date'] == date]
            # options_data = options_dict[ticker][date]
        except KeyError as e:
            print(f"KeyError fetching options data for {ticker} on {date}: {e}")
            continue
        except Exception as e:
            print(f"Error fetching options data for {ticker} on {date}: {e}")
            continue
        
        close_value = prices_eod_primary[(prices_eod_primary['ticker'] == ticker) & (prices_eod_primary['date'] == date)]
        
        try:
            h_value = get_h_value(ticker, date, close_value, options_data, False)  # Ensure `get_h_value` is defined
        except Exception as e:
            print(f"Error calculating H value for {ticker} on {date}: {e}")
            continue

        # Append the new row to the list
        h_values_list.append({'Ticker': ticker, 'Year': year, 'Date': date, 'Price': close_value['close'].iloc[0], 'H_value': h_value})

    return h_values_list

def main():
    # ticker_list = ['AMZN']
    # ticker_list = options_dict.keys()
    year_list = range(2010, 2024)  # Example: from 2020 to 2023

    # Generate all combinations of tickers and years
    args = [(ticker, year) for ticker in ticker_list for year in year_list]
    
    # Run the function in parallel for each combination
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_ticker_year)(ticker, year) for ticker, year in args
    )
    print(f"Number of active jobs running in parallel: {len(results)}")

    # Flatten the results into a single list
    h_values_list = [item for sublist in results for item in sublist]

    # Create the DataFrame from the list
    h_values_df = pd.DataFrame(h_values_list)

    return h_values_df

# Run the main function
h_values_df = main()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
h_values_df.sort_values(by='Date', inplace=True)

In [ ]:
h_values_df.reset_index(drop=True, inplace=True)

In [ ]:
# Ensure 'Date' is in datetime format
h_values_df['Date'] = pd.to_datetime(h_values_df['Date'])

# Resample using both 'Ticker' and 'Date' grouped by month
monthly_h_values_df = (
    h_values_df
    .groupby(['Ticker', pd.Grouper(key='Date', freq='M')])  # Resample each ticker by month-end
    .agg({
        'Price': 'last',  # Last price of the month
        'H_value': 'mean'  # Mean H-value for the month
    })
    .reset_index()
)




/var/folders/7c/fms9v7rj3w3_ssh2tnrr_0t00000gn/T/ipykernel_48885/381736551.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(['Ticker', pd.Grouper(key='Date', freq='M')])  # Resample each ticker by month-end


The above code has to be optimized, taking H values from a sample run

In [18]:
# Read the saved CSV file containing implied H values
implied_h_values = pd.read_csv('implied_H.csv')

# Convert Date column to datetime
implied_h_values['Date'] = pd.to_datetime(implied_h_values['Date'])

# Display the first few rows
implied_h_values.head()

,Ticker,Date,Price,H_value
0,APD,2010-01-31,75.96,0.215457
1,PFE,2010-01-31,18.66,-0.378224
2,WMB,2010-01-31,20.84,-0.072494
3,DLR,2010-01-31,50.15,-0.178283
4,VZ,2010-01-31,29.33,-0.176748


## Exploratory Analysis on the Implied H values

In [ ]:
analysis_industries=['US Consumer Discretionary', 'US Consumer Staples','US Energy', 'US Financials', 'US Healthcare']

In [29]:
# Merge industries dataframe with implied_h_values to get industry information
merged_data = pd.merge(implied_h_values, industries, left_on='Ticker', right_on='tickers', how='inner')

# Filter for the analysis industries
filtered_data = merged_data[merged_data['industry'].isin(analysis_industries)]

# Group by industry and calculate mean H_value
industry_h_values = filtered_data.groupby('industry')['H_value'].agg(['mean', 'std', 'count']).round(3)

# Sort by mean H_value
industry_h_values = industry_h_values.sort_values('mean', ascending=False)

print("Industry-wise H-value statistics:")
industry_h_values

Industry-wise H-value statistics:


,mean,std,count
industry,,,
US Financials,0.061,0.110,40
US Energy,0.044,0.181,29
US Consumer Staples,-0.039,0.264,9
US Healthcare,-0.079,0.240,231
US Consumer Discretionary,-0.100,0.176,74


## Limited Sample Size Effect
- The results may not be fully conclusive due to **insufficient data points** in some industries (e.g., **Consumer Staples: 9**, **Energy: 29**).
- A **larger dataset** would provide more reliable estimates of industry-specific behavior.

## Mean-Reverting Industries (Lower Hurst Exponent, \( H \))
- **US Consumer Staples, Healthcare, and Consumer Discretionary** tend to be **more stable over time**, meaning their stock prices **exhibit mean reversion**.
- This suggests that **price movements are short-lived** and tend to **revert to the mean** rather than persisting.
- These industries have a **lower \( H \)**, indicating **lower roughness and shorter memory in price movements**.

## Financials & Energy: Longer Memory, Yet Lower \( H \)
- **US Financials and Energy** also have a **low \( H \)** but **positive means**, indicating their prices have **longer memory** compared to the above industries.
- This suggests that **shocks or trends in these sectors persist longer** before reverting.
- **Financials and Energy** tend to be **more cyclical and macro-sensitive**, leading to a different return structure.

